Case Studies

Project: 1

Group: 3

Group Members:
 - Muhammad Raafey Tariq (231806)
 - Farrukh Ahmed (230614)
 - Amirreza Khamehchin Khiabani (230891)
 - Aymane Hachcham (236392)


Requirements:
 - numpy==1.24.2
 - matplotlib==3.7.1
 - seaborn==0.12.2
 - pandas==2.0.0
 - openpyxl==3.1.2

Installation Commands (One-time only)
 - pip install pandas==2.0.0
 - pip install numpy==1.24.2
 - pip install seaborn==0.12.2
 - pip install matplotlib==3.7.1
 - pip install openpyxl==3.1.2

Imports and Libraries

In [150]:
import pandas as pd
import numpy as np
import itertools
import pprint

# used for the graphs
import seaborn as sns
import os
sns.set(font_scale = 1.2)

# used for plotting
from matplotlib import pyplot as plt
from datetime import datetime
import matplotlib

# setting font to 'Times New Roman'
matplotlib.rcParams["font.family"] = "Times New Roman"
matplotlib.rcParams.update({'font.size': 16})
%matplotlib inline

Global Variables and Constants

Importing Data

In [151]:
file_path = "../styrian_health_data.xlsx"
sheet_name = "Sheet 1"
data_df = pd.read_excel(file_path, sheet_name=sheet_name)

c:\Users\Raafe\AppData\Local\Programs\Python\Python38\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


Reformating Columns to Correct Data Types

In [152]:
data_df.postleitzahl = data_df.postleitzahl.astype('Int64')
data_df.geburtsjahr = data_df.geburtsjahr.astype('Int64')
data_df.befinden = data_df.befinden.astype('Int64')
data_df.messwert_bp_sys = pd.to_numeric(data_df.messwert_bp_sys)
data_df.messwert_bp_dia = pd.to_numeric(data_df.messwert_bp_dia)
data_df.schaetzwert_bp_sys = pd.to_numeric(data_df.schaetzwert_bp_sys)
data_df.schaetzwert_by_dia = pd.to_numeric(data_df.schaetzwert_by_dia)

# adding variable for age
age =  data_df["zeit"].dt.year - pd.to_datetime(data_df['geburtsjahr'], format='%Y').dt.year
data_df["age"] = age.astype("Int64")

data_df["age_group"] = pd.cut(data_df.age, bins=[0,12,19,65,130],labels=['children', 'teenager', 'adult','65 over'])

In [153]:
print("Number of datapoints: ", len(data_df))
print("Number of variables: ", len(data_df.columns))
data_df.columns

Number of datapoints:  16386
Number of variables:  20


Index(['id', 'zeit', 'terminal', 'postleitzahl', 'gemeinde', 'bezirk',
       'bundesland', 'befinden', 'geburtsjahr', 'geschlecht', 'raucher',
       'blutzucker_bekannt', 'cholesterin_bekannt', 'in_behandlung',
       'schaetzwert_bp_sys', 'schaetzwert_by_dia', 'messwert_bp_sys',
       'messwert_bp_dia', 'age', 'age_group'],
      dtype='object')

In [154]:
data_df.isna().sum()

id                       0
zeit                     0
terminal                 0
postleitzahl           334
gemeinde               331
bezirk                 331
bundesland             331
befinden                23
geburtsjahr             23
geschlecht              23
raucher                  0
blutzucker_bekannt       0
cholesterin_bekannt      0
in_behandlung            0
schaetzwert_bp_sys      45
schaetzwert_by_dia      56
messwert_bp_sys          0
messwert_bp_dia          0
age                     23
age_group               26
dtype: int64

Investigating false data

In [155]:
max_age = data_df["age"].max()
print("Maximum age: ", max_age)

min_age = data_df["age"].min()
print("Minimum age: ", min_age)

Maximum age:  126
Minimum age:  0


In [156]:
children = data_df[data_df["age"] <= 12]
children.shape

(803, 20)

In [157]:
possible_false_data_1 = children[children.blutzucker_bekannt | children.cholesterin_bekannt | children.raucher | children.in_behandlung]
possible_false_data_1.shape

(45, 20)

In [158]:
possible_false_data_1.head()

,id,zeit,terminal,postleitzahl,gemeinde,bezirk,bundesland,befinden,geburtsjahr,geschlecht,raucher,blutzucker_bekannt,cholesterin_bekannt,in_behandlung,schaetzwert_bp_sys,schaetzwert_by_dia,messwert_bp_sys,messwert_bp_dia,age,age_group
849,12545,2006-05-09 10:51:53,1,8052,Graz - Wetzelsdorf,Graz(Stadt),Steiermark,2,1994,f,False,True,True,False,120.0,80.0,92,71,12,children
905,12605,2006-05-10 09:50:08,3,8054,Seiersberg,Graz-Umgebung,Steiermark,1,1994,m,True,False,False,True,75.0,70.0,112,70,12,children
955,12656,2006-05-10 16:52:37,1,1230,Wien,Wien,Wien,5,1999,f,True,True,True,True,190.0,80.0,143,124,7,children
1053,12791,2006-05-12 17:25:27,2,8051,Graz - Gösting,Graz(Stadt),Steiermark,1,2003,m,False,False,True,False,120.0,80.0,81,64,3,children
1226,13043,2006-05-14 12:56:53,2,8200,Nitscha,Weiz,Steiermark,2,1994,f,False,False,True,False,115.0,75.0,139,97,12,children


In [172]:
data_df[data_df["age"] >= 110]

,id,zeit,terminal,postleitzahl,gemeinde,bezirk,bundesland,befinden,geburtsjahr,geschlecht,raucher,blutzucker_bekannt,cholesterin_bekannt,in_behandlung,schaetzwert_bp_sys,schaetzwert_by_dia,messwert_bp_sys,messwert_bp_dia,age,age_group
18,11551,2006-04-27 15:52:15,1,<NA>,NaN,NaN,NaN,4,1880,f,False,False,False,False,180.0,70.0,123,87,126,65 over
30,11566,2006-04-27 17:40:52,3,1180,Wien,Wien,Wien,3,1880,f,False,False,False,False,NaN,NaN,125,95,126,65 over
301,11899,2006-04-30 16:09:38,3,1010,Wien,Wien,Wien,3,1880,m,False,False,False,False,100.0,80.0,121,74,126,65 over
1283,13119,2006-05-14 15:19:57,2,8720,Knittelfeld,Knittelfeld,Steiermark,5,1889,m,True,False,False,True,299.0,40.0,115,77,117,65 over
3431,16707,2006-06-07 15:47:07,2,1120,Wien,Wien,Wien,3,1880,f,True,False,True,False,198.0,78.0,137,84,126,65 over
12033,28484,2006-09-14 10:30:54,1,<NA>,NaN,NaN,NaN,5,1881,m,True,False,True,True,146.0,88.0,105,61,125,65 over
14908,32305,2006-10-17 11:08:30,2,<NA>,NaN,NaN,NaN,5,1880,f,True,True,True,True,123.0,88.0,141,79,126,65 over
16197,33953,2006-10-27 12:38:09,1,<NA>,NaN,NaN,NaN,5,1889,m,True,False,False,False,117.0,85.0,121,88,117,65 over


In [173]:
data_df[data_df["age"] == 0]

,id,zeit,terminal,postleitzahl,gemeinde,bezirk,bundesland,befinden,geburtsjahr,geschlecht,raucher,blutzucker_bekannt,cholesterin_bekannt,in_behandlung,schaetzwert_bp_sys,schaetzwert_by_dia,messwert_bp_sys,messwert_bp_dia,age,age_group
14907,32304,2006-10-17 11:07:33,3,<NA>,NaN,NaN,NaN,5,2006,f,True,True,True,True,112.0,90.0,113,90,0,NaN
15109,32745,2006-10-20 10:34:21,1,8510,Stallhof,Deutschlandsberg,Steiermark,5,2006,m,True,True,True,True,299.0,160.0,118,74,0,NaN
16042,33750,2006-10-26 13:55:15,2,8713,Sankt Stefan ob Leoben,Leoben,Steiermark,5,2006,m,True,False,False,True,99.0,98.0,72,60,0,NaN


 - debate whether or not these should be excluded
 - how old is too old? more than 100
 - is it likely that children less than age of 12 have diabetes, cholestrol, receive hyper tension treatment?
 - should all children less than 5 or 10 be removed? how is the measurement even made for children
 - children that smoke definetly should be removed 

Investigating IDs

In [160]:
unique_id_list = data_df["id"].unique()
str_ids = [str(id) for id in unique_id_list]

In [169]:
# observing increasing pattern
count = 0
diff_list = []
for i in range(1, len(unique_id_list)):
    curr_id = unique_id_list[i-1]
    next_id = unique_id_list[i]
    diff = next_id - curr_id

    if diff > 1:
        diff_list.append(diff)
        # print("Diff between ", curr_id, " and ", next_id, " at indices ", i-1, " and ", i)
        count += 1

print("Num breaks in id: ", count)
np.unique(diff_list)

Num breaks in id:  3095


array([  2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,  14,
        15,  16,  17,  18,  19,  20,  22,  25,  26,  27,  29,  38,  55,
        59, 101, 106, 154, 161], dtype=int64)

 - Id breaks off quite a few times with huge jumps at some moments
 - Still ids do tend to increase, sometimes regular, sometimes irregular
 - It is possible that some users opted to have their data not published hence missing ids
 - that data might have been corrupted or lost before or when producing the xlsx sheet
 - Unclear why id does not start with 0: perhaps the terminals used for this study have been used elsewhere before at some other time for some other purpose

Outside Measurements

In [162]:
start_time = datetime(2006, 4, 29)
end_time = datetime(2006, 10, 30)

def filter_fn(row):
    status = row["zeit"] >= start_time and row["zeit"] < end_time
    return status

mask = data_df.apply(filter_fn, axis=1)
outside_survey_df = data_df[~mask]

before_survey_df = outside_survey_df[outside_survey_df["zeit"] < start_time] 
after_survey_df = outside_survey_df[outside_survey_df["zeit"] >= end_time] 
outside_survey_df.to_csv("temp.csv")

In [163]:
print("Total surveys conducted outside data collection window: ", outside_survey_df.shape)
print("Total surveys conducted before data collection window: ", before_survey_df.shape)
print("Total surveys conducted after data collection window: ", after_survey_df.shape)

Total surveys conducted outside data collection window:  (135, 20)
Total surveys conducted before data collection window:  (132, 20)
Total surveys conducted after data collection window:  (3, 20)


In [164]:
# checking quality of this subset

outside_survey_df.isna().sum()

id                      0
zeit                    0
terminal                0
postleitzahl           10
gemeinde               10
bezirk                 10
bundesland             10
befinden                0
geburtsjahr             0
geschlecht              0
raucher                 0
blutzucker_bekannt      0
cholesterin_bekannt     0
in_behandlung           0
schaetzwert_bp_sys      2
schaetzwert_by_dia      3
messwert_bp_sys         0
messwert_bp_dia         0
age                     0
age_group               0
dtype: int64

 - Unclear why measurements were made before or after
 - Data doesn't have much missing values
 - Are the actual start and end dates different?
 - Alot of readings, consistently throughout the day on 27th means terminals were operational - maybe just a beta test before actual survey
 - only 3 readings after end date